# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import patsy 
import statsmodels.api as sm
#import statsmodels.formula.api as smf

%matplotlib inline 

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   index                  15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12466 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  mau                    15000 non-null  bool   
 15  re

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

### 1.Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
Utilizando os recursos do Patsy, coloque as variáveis qualitativas como dummies.
Mantenha sempre a categoria mais frequente como casela de referência
Avalie os parâmetros e veja se parecem fazer sentido prático.

In [4]:
#Como tempo de emprego tem informações não preenchidas vamos dropar os registros e deletar variáveis não desejadas para o modelo 
df = df.dropna()
df.drop(columns=['Unnamed: 0','data_ref','index'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12466 entries, 0 to 14999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   12466 non-null  object 
 1   posse_de_veiculo       12466 non-null  bool   
 2   posse_de_imovel        12466 non-null  bool   
 3   qtd_filhos             12466 non-null  int64  
 4   tipo_renda             12466 non-null  object 
 5   educacao               12466 non-null  object 
 6   estado_civil           12466 non-null  object 
 7   tipo_residencia        12466 non-null  object 
 8   idade                  12466 non-null  int64  
 9   tempo_emprego          12466 non-null  float64
 10  qt_pessoas_residencia  12466 non-null  float64
 11  mau                    12466 non-null  bool   
 12  renda                  12466 non-null  float64
dtypes: bool(3), float64(3), int64(2), object(5)
memory usage: 1.1+ MB


In [5]:
#Regressão.
y, X = patsy.dmatrices('np.log(renda) ~  sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao) + C(estado_civil) + C(tipo_residencia) + idade + tempo_emprego + qt_pessoas_residencia + mau', df)
modelo = sm.OLS(y,X).fit()
modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.242
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     158.7
Date:                Mon, 23 May 2022   Prob (F-statistic):               0.00
Time:                        10:04:49   Log-Likelihood:                -13204.
No. Observations:               12466   AIC:                         2.646e+04
Df Residuals:                   12440   BIC:                         2.665e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               7.2007      0.272     26.484      0.000       6.668       7.734
sexo[T.M]                               0.4815      0.014     33.681      0.000       0.453       0.510
posse_de_veiculo[T.True]               -0.0014      0.014     -0.104      0.917      -0.028       0.026
posse_de_imovel[T.True]                 0.0847      0.014      6.260      0.000       0.058       0.111
C(tipo_renda)[T.Bolsista]              -0.0499      0.350     -0.143      0.887      -0.735       0.636
C(tipo_renda)[T.Empresário]             0.1787      0.015     12.217      0.000       0.150       0.207
C(tipo_renda)[T.Pensionista]            0.4099      0.237      1.732      0.083      -0.054       0.874
C(tipo_renda)[T.Servidor público]       0.0949      0.021      4.482      0.000       0.053       0.136
C(educacao)[T.Pós graduação]            0.5496      0.175      3.133      0.002       0.206       0.893
C(educacao)[T.Secundário]               0.0166      0.071      0.233      0.816      -0.123       0.156
C(educacao)[T.Superior completo]        0.1134      0.071      1.590      0.112      -0.026       0.253
C(educacao)[T.Superior incompleto]     -0.0090      0.076     -0.117      0.907      -0.159       0.141
C(estado_civil)[T.Separado]            -0.0395      0.131     -0.302      0.763      -0.296       0.217
C(estado_civil)[T.Solteiro]             0.0213      0.129      0.166      0.868      -0.231       0.273
C(estado_civil)[T.União]               -0.0256      0.023     -1.098      0.272      -0.071       0.020
C(estado_civil)[T.Viúvo]               -0.0377      0.134     -0.280      0.779      -0.301       0.226
C(tipo_residencia)[T.Casa]              0.0146      0.053      0.274      0.784      -0.090       0.119
C(tipo_residencia)[T.Com os pais]      -0.0578      0.060     -0.967      0.334      -0.175       0.059
C(tipo_residencia)[T.Comunitário]       0.0571      0.103      0.555      0.579      -0.145       0.259
C(tipo_residencia)[T.Estúdio]          -0.1093      0.089     -1.231      0.218      -0.283       0.065
C(tipo_residencia)[T.Governamental]    -0.0567      0.064     -0.880      0.379      -0.183       0.070
mau[T.True]                            -0.0323      0.042     -0.778      0.437      -0.114       0.049
qtd_filhos                              0.0047      0.128      0.037      0.971      -0.247       0.256
idade                                   0.0052      0.001      6.628      0.000       0.004       0.007
tempo_emprego                           0.0482      0.001     48.349      0.000       0.046       0.050
qt_pessoas_residencia                   0.0114      0.128      0.089      0.929      -0.239       0.262
===================================================================

In [6]:
#usando o groupby para entender a frequência das variáveis dummies
df.groupby('tipo_renda')['renda'].count()
df.groupby('educacao')['renda'].count()
df.groupby('estado_civil')['renda'].count()
df.groupby('tipo_residencia')['renda'].count()

tipo_residencia
Aluguel            176
Casa             11128
Com os pais        636
Comunitário         63
Estúdio             97
Governamental      366
Name: renda, dtype: int64

In [7]:
#Regressão.
y, X = patsy.dmatrices('np.log(renda) ~  sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao,Treatment(2)) + C(estado_civil) + C(tipo_residencia) + idade + tempo_emprego + qt_pessoas_residencia + mau', df)
modelo = sm.OLS(y,X).fit()
modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.242
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     158.7
Date:                Mon, 23 May 2022   Prob (F-statistic):               0.00
Time:                        10:04:49   Log-Likelihood:                -13204.
No. Observations:               12466   AIC:                         2.646e+04
Df Residuals:                   12440   BIC:                         2.665e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            7.2172      0.263     27.461      0.000       6.702       7.732
sexo[T.M]                                            0.4815      0.014     33.681      0.000       0.453       0.510
posse_de_veiculo[T.True]                            -0.0014      0.014     -0.104      0.917      -0.028       0.026
posse_de_imovel[T.True]                              0.0847      0.014      6.260      0.000       0.058       0.111
C(tipo_renda)[T.Bolsista]                           -0.0499      0.350     -0.143      0.887      -0.735       0.636
C(tipo_renda)[T.Empresário]                          0.1787      0.015     12.217      0.000       0.150       0.207
C(tipo_renda)[T.Pensionista]                         0.4099      0.237      1.732      0.083      -0.054       0.874
C(tipo_renda)[T.Servidor público]                    0.0949      0.021      4.482      0.000       0.053       0.136
C(educacao, Treatment(2))[T.Primário]               -0.0166      0.071     -0.233      0.816      -0.156       0.123
C(educacao, Treatment(2))[T.Pós graduação]           0.5330      0.161      3.316      0.001       0.218       0.848
C(educacao, Treatment(2))[T.Superior completo]       0.0969      0.013      7.200      0.000       0.070       0.123
C(educacao, Treatment(2))[T.Superior incompleto]    -0.0255      0.031     -0.826      0.409      -0.086       0.035
C(estado_civil)[T.Separado]                         -0.0395      0.131     -0.302      0.763      -0.296       0.217
C(estado_civil)[T.Solteiro]                          0.0213      0.129      0.166      0.868      -0.231       0.273
C(estado_civil)[T.União]                            -0.0256      0.023     -1.098      0.272      -0.071       0.020
C(estado_civil)[T.Viúvo]                            -0.0377      0.134     -0.280      0.779      -0.301       0.226
C(tipo_residencia)[T.Casa]                           0.0146      0.053      0.274      0.784      -0.090       0.119
C(tipo_residencia)[T.Com os pais]                   -0.0578      0.060     -0.967      0.334      -0.175       0.059
C(tipo_residencia)[T.Comunitário]                    0.0571      0.103      0.555      0.579      -0.145       0.259
C(tipo_residencia)[T.Estúdio]                       -0.1093      0.089     -1.231      0.218      -0.283       0.065
C(tipo_residencia)[T.Governamental]                 -0.0567      0.064     -0.880      0.379      -0.183       0.070
mau[T.True]                                         -0.0323      0.042     -0.778      0.437      -0.114       0.049
qtd_filhos                                           0.0047      0.128      0.037      0.971      -0.247       0.256
idade                                    

Avaliação: Mesmo mudando a casela de referência para a mais frequente os valores de R2, R2 ajustado e AIC não sofreram atualizações. Só precisei acertar a cancela de referência da variável 'educacao' as demais o modelo já havia colocado na variável que aparece com mais frequência. Analisando os parâmetros principalmente o P-values podemos verificar que existem várias variáveis que poderiam ser eliminadas do modelo.

### Remova a variável menos significante e analise:
Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
Observe os parâmetros e veja se algum se alterou muito.

In [8]:
#Regressão.
y, X = patsy.dmatrices('np.log(renda) ~  sexo  + posse_de_imovel  + idade + tempo_emprego ', df)
modelo = sm.OLS(y,X).fit()
modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.225
Method:                 Least Squares   F-statistic:                     907.7
Date:                Mon, 23 May 2022   Prob (F-statistic):               0.00
Time:                        10:04:50   Log-Likelihood:                -13335.
No. Observations:               12466   AIC:                         2.668e+04
Df Residuals:                   12461   BIC:                         2.672e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   7.3732      0.030    243.856      0.000       7.314       7.432
sexo[T.M]                   0.4742      0.013     35.424      0.000       0.448       0.500
posse_de_imovel[T.True]     0.1018      0.013      7.649      0.000       0.076       0.128
idade                       0.0043      0.001      5.958      0.000       0.003       0.006
tempo_emprego               0.0479      0.001     48.283      0.000       0.046       0.050
==============================================================================
Omnibus:                        1.296   Durbin-Watson:                   2.008
Prob(Omnibus):                  0.523   Jarque-Bera (JB):                1.272
Skew:                           0.011   Prob(JB):                        0.529
Kurtosis:                       3.045   Cond. No.                         207.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

A Avaliação dos parâmetros mostra que os indicadores R2 e R2 ajustado reduziram em aproximadamente 0,02 e o AIC aumentou em 220.000 quando fizemos uma redução das variáveis menos significantes (p-values > 0,05). No contexto geral, os parâmetros de R2, R2 ajustado e AIC mostram que é factível simplificarmos o modelo sem perdas consideráveis.  

### 3.Siga removendo as variáveis menos significantes, sempre que o p-value for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.

Já eliminei todos os valore acima de 0,05 de p-value, podemos observar que o modelo já ficou o mais otimizado possível considerando estas técnicas.

### Exemplo utilizando uma das caselas com patsy

In [10]:
#Regressão.
y, X = patsy.dmatrices('np.log(renda) ~  sexo  +C(tipo_renda,Treatment("Empresário"))+ posse_de_imovel  + idade + tempo_emprego ', df)
modelo = sm.OLS(y,X).fit()
modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.236
Model:                            OLS   Adj. R-squared:                  0.236
Method:                 Least Squares   F-statistic:                     481.8
Date:                Mon, 23 May 2022   Prob (F-statistic):               0.00
Time:                        10:06:05   Log-Likelihood:                -13249.
No. Observations:               12466   AIC:                         2.652e+04
Df Residuals:                   12457   BIC:                         2.658e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================================================================
                                                                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------
Intercept                                                      7.4854      0.031    237.701      0.000       7.424       7.547
sexo[T.M]                                                      0.4819      0.013     36.182      0.000       0.456       0.508
C(tipo_renda, Treatment("Empresário"))[T.Assalariado]         -0.1866      0.015    -12.806      0.000      -0.215      -0.158
C(tipo_renda, Treatment("Empresário"))[T.Bolsista]            -0.1653      0.351     -0.471      0.637      -0.853       0.522
C(tipo_renda, Treatment("Empresário"))[T.Pensionista]          0.2471      0.234      1.057      0.291      -0.211       0.706
C(tipo_renda, Treatment("Empresário"))[T.Servidor público]    -0.0784      0.023     -3.390      0.001      -0.124      -0.033
posse_de_imovel[T.True]                                        0.1009      0.013      7.625      0.000       0.075       0.127
idade                                                          0.0045      0.001      6.239      0.000       0.003       0.006
tempo_emprego                                                  0.0484      0.001     48.558      0.000       0.046       0.050
==============================================================================
Omnibus:                        0.874   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.646   Jarque-Bera (JB):                0.845
Skew:                          -0.004   Prob(JB):                        0.656
Kurtosis:                       3.039   Cond. No.                     2.38e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.38e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""